# <center>Informatique tc3 (Projet Web) - TD1</center>

## <center style="color: #66d">Projet 3 - Editeur de scène.</center>

### 1. Cahier des charges

<div>
<br>
Le module <tt>raytracer</tt> fourni pour les projets ne comporte que très peu d'objets composés (Cube, Cylindre, Cône tronqué). L'objectif de ce projet est de :
<ul style="margin-top:0">
 <li style="margin-top:0">créer de nouveaux objets composés comme des polyèdres (pyramide, maison, ...), ou des arbres (sphère + cylindre, cône + cylindre),
 <li style="margin-top:0">concevoir une interface permettant de créer des scènes comportant ce type d'objets. 
 </ul>
 </div>
<p>
<div>
L'interface permettra :
<ul style="margin-top:0">
 <li style="margin-top:0">d'ajouter ou de retirer des objets de la scène,
 <li style="margin-top:0">de définir la position, et l'orientation des objets,
 <li style="margin-top:0">la taille de l'image générée.
</ul>
</div>
 
Les images seront générées à la volée ou sur demande. Elles devront pouvoir être consultées sans être regénérées si elles existent déjà pour un choix de paramètres donné _(notion de cache serveur)_.

Pour des raisons de simplicité, les couleurs des objets pourront être fixées (i.e. non modifiables). De même, il ne sera pas forcément nécessaire de pouvoir modifier la position de la caméra, et les sources d'éclairage pourront être prédéfinies.

### 2. Exemple : Pyramide

La classe _Polygon2_ permet de créer un polygone à partir des coordonnées 3D de ses sommets. &Agrave; partir de là, il est assez facile de concevoir une pyramide générique, avec un sommet et une base composée d'un polygone quelconque :

In [1]:
from raytracer import *

class Pyramid(ComposedArtifact):

    # base est un Polygone
    # S est un vec3 correspondant au sommet
    def __init__(self, base, S, *args, **kwargs):

        # La pyramide est composée de n+1 facettes
        npoints = len(base.vertices)
        ComposedArtifact.__init__(self, npoints+1, *args, **kwargs)

        # La première facette de la pyramide est sa base
        self[0] = base
        
        # Si la base est un Polygon2 on dispose déjà des coordonnées 3D
        # des sommets, sinon on les calcule
        vertices3D = base.vertices3D if hasattr(base,'vertices3D') else \
          [base.P + base.U * v[0] + base.V * v[1] for v in base.vertices]

        # Chacune des facettes s'appuie sur le sommet et un segment de la base
        for n in range(npoints):
            P1 = S
            P2 = (vertices3D[n-1] if n > 0 else vertices3D[npoints-1])
            P3 = vertices3D[n]
            self[n+1] = Polygon2((P1,P2,P3), -base.ns, **self.kwargs(n))

        self.vertices3D = [*vertices3D, S]
        self.base = base
        self.S = S

    @classmethod
    def keys(cls):
        return ['base', 'S'] + Artifact.keys()

    def getattr(self,k):
        return getattr(self,k) if k in self.keys() else ComposedArtifact.getattr(self,k)

In [2]:
scene = Scene('pyramid', 0, 1)
scene.append(LightSource(vec3(-1., 2, 0), 1))

S = vec3(0,0.5,0.5)
vertices = [vec3(-0.5,0,0), vec3(0.5,0,0), vec3(0.5,0,1), vec3(-0.5,0,1)]
base = Polygon2(vertices, 1, diffuse=rgb(0.5,0.5,0), mirror=0, specular=0)

scene.append(Plane(vec3(0,-0.1,0), vec3(0,1,0)))
scene.append(Pyramid(base, S, diffuse=rgb(0.9,0.5,0.1), mirror=0, specular=0))

scene.initialize(E=vec3(0,2,-10)).trace().save_image()
pass

pyramid took 0.390s


<img src="pyramid.png" width="300">
<center><a href="pyramid.png">pyramid.png</a></center>